# Quantum Phase Estimation in Q\#

In this notebook, we will implement the QPE for single qubit unitaries (such as Z, S, and T). Following are a set of libraries that are useful for this assignment. 

In [1465]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;

## Task 1 - Quantum Fourier Transform 
Implement an **operation** ```QuantumFourierTransform``` that performs QFT on given set of input qubits. This operation should be **Adjointable** and **Controllable**. 

Args: 
    
    qs (Qubit[]): An array of qubits 

Returns: 

    None

In [1471]:
operation QuantumFourierTransform(qs: Qubit[]): Unit is Adj + Ctl{
    for i in 0 .. Length(qs) - 1 {

        H(qs[i]);

        if(i != (Length(qs)-1) ){

            for j in (i + 1) .. Length(qs) - 1 {
                //Apply U_ROTk
                let k = 1.0 + IntAsDouble(j-i);
                Controlled R1([qs[j]], (2.0*PI()/PowD(2.0, k), qs[i]));
            }
        }
    }
    for i in 0 .. Length(qs) - 1 {
        if(i != (Length(qs) - 1 - i)) {
            SWAP(qs[i], qs[Length(qs) - 1 - i]);
        }
    }
    
}


QuantumFourierTransform

## Task 2 - Controlled Unitaries for phase kickback

Implement an **operation** ```ControlledUnitaries``` that performs controlled rotations on given set of input qubits according to the Quantum Phase Estimation algorithm.

Args: 
    
    estimation_qs (Qubit[]): An array of qubits (controls)
    U (Qubit => Unit is Ctl + Adj): A single qubit unitary matrix
    target_qubit (Qubit): Target qubit for the controlled unitaries

Returns: 

    None
    
Hint: Checkout the ```Canon``` library to apply powers of $U$ easily. Note that you can do this without the library as well. 

In [1467]:
operation ControlledUnitaries(estimation_qs: Qubit[], U: Qubit => Unit is Ctl + Adj, target_qubit: Qubit): Unit {
    for i in 0 .. Length(estimation_qs)-1 {
        let Upowered = OperationPowCA(U, Truncate(PowD(2.0, IntAsDouble(Length(estimation_qs) - 1 - i))));
        Controlled Upowered(estimation_qs, target_qubit);
    }
}

ControlledUnitaries

## Task 3 - Convert from fractional binary to decimal
Implement a **function** ```ThetaDecimal```that converts the phase from fractional binary to decimal according to the formula:

$$\theta = 0.\theta_1\theta_2..\theta_t \text{  in fractional binary} \\
\theta = \frac{1}{2^1} + \frac{1}{2^2} + \cdots \frac{1}{2^t}  \text{  in decimal}$$

Args: 
    
    theta_bin (Int[]): An array of bits (containing 0 or 1) in fractional binary. E.g. 0.110 would be represented as [1, 1, 0]
   
Returns: 

    Double: The phase value in decimal. 
    
Possibly useful: https://learn.microsoft.com/en-us/azure/quantum/user-guide/libraries/standard/convert

In [1468]:
function ThetaDecimal(theta_bin: Int[]): Double {
    mutable result = 0.0;
    for i in 0 .. Length(theta_bin)-1 {
        set result = result + (IntAsDouble(theta_bin[i])/PowD(2.0, IntAsDouble(i+1)));
    }
    
    return result;
}

ThetaDecimal

## Task 4 - QPE for Z, T, X
Use ***ONLY*** operations and functions from Task 1, Task 2, and Task 3 to calculate $\theta$ for the Z gate, T gate, X gate. For each of the cases answer the following questions:

1. What are the eigenvalues and eigenvectors of each of the gates? The Z gate is given. Fill in the following table. 
2. How many qubits are required for each of the cases?
3. What is the phase observed in each of the cases?

Fill in the following table to answer 1, 2, and 3.
  

I believe that there is a bug in my QFT as when I try and find the eigenvalue for the $|1\rangle$ with the T gate I get non deterministic results as the adjoing QFT doesn't set a state to have an amplitude of 1 (it does for every other case shown below, just not for the T gate with $|1\rangle$). I've commented DumpMachine calls so that you can see it if you'd like. Mathematically though the eigenvalue is $e^{\frac{\pi i}{4}}$, the min est qubits would be 3 (for 0.001$_2$ = 0.125$_{10}$) and the phase observed would be 0.125.

| Gate  	| Eigenvector  	| Eigenvalue 	| Min. est. qubits* | Phase observed^  	|
|-------	|--------------	|------------	|-----------------	|-----------------	|
| Z     	| $|0\rangle$ 	| 1          	| 1               	| 0.0              	|
|       	| $|1\rangle$ 	| -1         	| 1               	| 0.5              	|
| T     	| $|0\rangle$ 	| 1          	| 1               	| 0.0              	|
|       	| $|1\rangle$ 	|            	|                 	|               	|
| X     	| $|+\rangle$  	| 1           	| 1                	| 0.0              	|
|       	| $|-\rangle$  	| -1           	| 1                	| 0.5               |

\* = Min. est. qubits = Minimum Estimation qubits (not including the target qubit)

^ = Phase observed from the operation below 

Args: 
    
    None
   
Returns: 

    Double: The phase value obtained by running QPE

In [1598]:
operation QPE(): Double {
    use estimation_qs = Qubit[1]; 
    use target = Qubit();
    
    //prepare eigenstate
    
    X(target);
    //H(target);
    
    // Use *ONLY* operations/functions from Tasks 1, 2, 3
    // To implement QPE
    
    //apply hadamard basis to estimation qubits
    for q in estimation_qs{
        QuantumFourierTransform([q]);
    }
    //DumpMachine();
    ControlledUnitaries(estimation_qs, T, target);
    //DumpMachine();
    Adjoint QuantumFourierTransform(estimation_qs);
    DumpMachine();

    // Interpret results
    let decodedBits = ForEach((x) => M(x) == One ? 1 | 0, estimation_qs);
    Reset(target);
    return ThetaDecimal(decodedBits);
}

QPE, _474be78176664122a985fd8360726666_QPE

In [1627]:
%simulate QPE

|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.853553390593274 + 0.35355339059327384𝑖
|3⟩	0.14644660940672627 + -0.35355339059327384𝑖

0

## Task 5 - QPE for an arbitrary Z rotation

Find the phase of ```MysteryGate``` operation when the target qubit is prepared in the $|1\rangle$ state.

[Definition of the Rz gate](https://learn.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic.rz)

Answer the following question:
1. What is the $\theta$ value that we expect from QPE?
    * Given the below MysteryGate, we would expect the value of ${\large e^\frac{4i\pi}{3}}$ and so $\theta$ would be $\frac{2}{3}$  
    
2. What is the $\theta$ value that you get when number of estimation qubits are
    * 2: Again, my QPE is not working, but it should be 0.10
    * 5: Should be 0.10101
    * 10: Should be 0.1010101010
3. Explain your results.
    * $\frac{2}{3} = 0.66666...$ and so to represent that number in fractional binary, you happen to need alternating 1's and 0's, so it forms a pattern.

In [1628]:
operation MysteryGate(qubit: Qubit) : Unit is Adj+Ctl {
    Rz(4.0*PI()/3.0, qubit);
}

MysteryGate

In [1629]:
operation MysteryQPE(): Double {
    use estimation_qs = Qubit[1]; 
    use target = Qubit();
    
    //prepare eigenstate
    
    X(target);
    //H(target);
    
    // Use *ONLY* operations/functions from Tasks 1, 2, 3
    // To implement QPE
    
    //apply hadamard basis to estimation qubits
    for q in estimation_qs{
        QuantumFourierTransform([q]);
    }
    DumpMachine();
    ControlledUnitaries(estimation_qs, T, target);
    DumpMachine();
    Adjoint QuantumFourierTransform(estimation_qs);
    DumpMachine();

    // Interpret results
    let decodedBits = ForEach((x) => M(x) == One ? 1 | 0, estimation_qs);
    Reset(target);
    return ThetaDecimal(decodedBits);
}

MysteryQPE, _cd60ac42a13b4f3e8b2176730d66d40b_MysteryQPE

In [1639]:
%simulate MysteryQPE

|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.7071067811865476 + 0𝑖
|3⟩	0.5000000000000001 + 0.5000000000000001𝑖

|0⟩	0 + 0𝑖
|1⟩	0 + 0𝑖
|2⟩	0.853553390593274 + 0.35355339059327384𝑖
|3⟩	0.14644660940672624 + -0.35355339059327384𝑖

0